# AutoGenU for Jupyter

An Automatic C++ Code Generator for Nonlinear Model Predictive Control (NMPC)  

---  

# Generate C++ codes of NMPC model  

## Import modules

In [ ]:
from sympy import *
from AutoGenU import basic_symbolic_func as symfunc
from AutoGenU import cpp_generator as gencpp
init_printing()

## Set dimensions
`dimx`: dimension of the state vector $x$   
`dimu`: dimension of the control input vector $u$  
`dimc`: dimension of the constraints $C(x, u) = 0$  

In [ ]:
dimx = 4
dimu = 2
dimc = 1

## Generate t, x, u, and lmd, necessary variables for the formulation of the optimal control problem
`t`: time parameter $t$  
`x`: the state vector $x$  
`u`: the control input vector $u$  
`lmd`: the Lagrange multiplier vector for the state equation $\lambda$

In [ ]:
t = Symbol('t')
x = symbols(f'x[0:{dimx}]')
u = symbols(f'u[0:{dimu+dimc}]')
lmd = symbols(f'lmd[0:{dimx}]')

## Define variables used in the state equation, constraints, and cost function
Define a scalar variable whose name is "var" as  
`var = Symbol('var') `

Define scalar variables whose names are "var\_1", "var\_2", ..., "var\_n" as  
`var_1, var_2, ..., var_n = symbols('var_1, var_2, ..., var_n')`

Define a vector whose name is "vec" and dimension is dim_vec as  
`vec = symbols(f'vec[0:{dim_vec}]')`

In [ ]:
# Define user variables used in the state function here
m_c, m_p, l, g = symbols('m_c, m_p, l, g')

# variables for constraints on the control input
u_min, u_max, dummy_weight = symbols('u_min, u_max, dummy_weight')

## Define weight parameters used in the stage cost and the terminal cost
`q`: weight array for the state in the stage cost  
`r`: weight array for the control input in the stage cost  
`q_terminal`: weight array for the state in the terminal cost  
`x_ref`: the reference value of the state

In [ ]:
q = symbols(f'q[0:{dimx}]')
r = symbols(f'r[0:{dimu}]')
q_terminal = symbols(f'q_terminal[0:{dimx}]')
x_ref = symbols(f'x_ref[0:{dimx}]')

## Define the state equation, constraints, the stage cost, and the terminal cost
`fxu`: state equation $ f(t, x, u)$  
`Cxu`: equality constraisnts $C(x, u, t) $  
`L`: stage cost $L(x, u, t)$  
`phi`: terminal cost $\phi (x, t)$  
Note: array indices start with 0

In [ ]:
# Define the state equation
fxu = [x[2], 
       x[3],
      (u[0] + m_p*sin(x[1])*(l*x[1]*x[1] + g*cos(x[1])) )/( m_c+m_p*sin(x[1])*sin(x[1])),
      (-u[0] * cos(x[1]) - m_p*l*x[1]*x[1]*cos(x[1])*sin(x[1]) - (m_c+m_p)*g*sin(x[1]) )/( l*(m_c + m_p*sin(x[1])*sin(x[1])))]

# Define the constraints (if dimc > 0)
Cxu = [u[0]**2 + u[1]**2 - ((u_max-u_min)**2)/4]

# Define the stage cost
L = sum(q[i]*(x[i] - x_ref[i])**2 for i in range(dimx))/2 + (r[0] * u[0]**2)/2 - dummy_weight*u[1] 

# Define the terminal cost
phi = sum(q_terminal[i]*(x[i] - x_ref[i])**2 for i in range(dimx))/2 

## Generate the optimality conditions
`hamiltonian`: $H(x, u, \lambda) = L(x, u) + \lambda^{\rm T} f(x, u)$  
`hx`: partial derivartive of the hamiltonian $H (x, u, \lambda)$ with respect to $x$, $\left(\frac{\partial H}{\partial x} \right)^{\rm T} (x, u, \lambda)$    
`hu`: partial derivartive of the hamiltonian $H (x, u, \lambda)$ with respect to $u$, $\left(\frac{\partial H}{\partial u} \right)^{\rm T}, (x, u, \lambda)$    
`phix`: partial derivative of the terminal cost $\phi(x)$ with respect to $x$, $\left(\frac{\partial \phi}{\partial x} \right)^{\rm T} (x)$

In [ ]:
if(dimc > 0):
    hamiltonian = L + symfunc.dotProduct(lmd, fxu) + sum(u[dimu+i] * Cxu[i] for i in range(dimc))
else:
    hamiltonian = L + symfunc.dotProduct(lmd, fxu) 

phix = symfunc.diffScalarFunc(phi, x)
hx = symfunc.diffScalarFunc(hamiltonian, x)
hu = symfunc.diffScalarFunc(hamiltonian, u)

## Symplify phix, hx, and hu
Note: if `hx` and `hu` is too complicated, it takes too much time to simplify them

In [ ]:
phix = simplify(phix)
hx = simplify(hx)
hu = simplify(hu)

## Set Parameters
set all parameters used in the state equation, constraints, and the cost function

In [ ]:
# scalar parameters
scalar_params = [[m_c, 2], [m_p, 0.2], [l, 0.5], [g, 9.80665], 
                 [u_min, -15], [u_max, 15], [dummy_weight, 0.1]]

# array parameters
array_params = [['q', dimx, '{0.1, 10, 0.01, 0.01}'], 
                ['r', dimu, '{1, 0.01}'], 
                ['q_terminal', dimx, '{0.1, 10, 0.01, 0.01}'], 
                ['x_ref', dimx, '{0, M_PI, 0, 0}']]


## Generate C++ codes of NMPC model
generate `nmpc_model.hpp` and `nmpc_model.cpp` in a directory of `model_name`

In [ ]:
model_name = "cartpole"

gencpp.makeModelDir(model_name)
gencpp.generateCpp(dimx, dimu, dimc, fxu, Cxu, phix, hx, hu, model_name)
gencpp.generateHpp(dimx, dimu, dimc, scalar_params, array_params, model_name)

---  
# Generate C++ codes for numerical simulation  

## Import modules

In [ ]:
from AutoGenU import solver_params as solverparams
from AutoGenU import simulation_params as simparams
from AutoGenU import cpp_executor as cppexec
from AutoGenU import simulation_plottor as simplot
from AutoGenU import animation_generator as animgen

## Set solvers  
set which solvers you use  in `solver_index`
  
1. The continuation/GMRES method (the original C/GMRES method, single shooting)
2. The multiple shooting based C/GMRES method
3. The multiple shooting based C/GMRES method with condensing of variables with respect to the constraints on the saturation function on the control input

In [ ]:
solver_index = 1

## Set saturaions on the control input if you choose `solver_index = 3`
- saturation on the control input: $u_{i, {\rm min}} \leq u_i \leq u_{i, {\rm max}}$  
$u_i \in \mathbb{R}$ : a constrained component of the control input $u$  
- transformed equality constraint: $(u_i - \frac{u_{i, {\rm max}} + u_{i, {\rm min}}}{2})^2 - ( \frac{u_{i, {\rm max}} - u_{i, {\rm min}}}{2})^2 + {u_d}_i ^2 = 0$  
${u_d}_i \in \mathbb{R}$ : a dummy input for the transformation of the saturation into the equality constraint  
- additional term in the stage cost $L(x, u)$ with respect to the saturation of $u_i$: $- {r_d}_i {u_d}_i + \frac{1}{2} {r_q}_i {u_d}_i ^2$  
   ${r_d}_i > 0$: a weight parameter to avoid failure of numerical computation, ${r_q}_i \geq 0$: a weight parameter to increase mergin of the saturation  

`index`: $i$  
`u_min`: $u_{i, {\rm min}}$  
`u_max`: $u_{i, {\rm max}}$   
`dummy_weight`: ${r_d}_i > 0$  
`quadratic_weight` :  ${r_q}_i \geq 0$  
`saturation_param` = [`index`, `u_min`, `u_max`, `dummy_weight`, `quadratic_weight`]

In [ ]:
saturation_list = []
# saturation_list = [[index , u_min, u_max, dummy_weight, quadratic_weight], 
#                    [index , u_min, u_max, dummy_weight, quadratic_weight], 
#                    .., 
#                    [index , u_min, u_max, dummy_weight, quadratic_weight]

## Set parameters for the solver

`T_f`, `alpha`: parameters for the length of the horizon $T(t)$: $T(t) = T_f (1 - e^{-\alpha t})$  
`horizon_division_num`: the division number of the horzion for numerical computation  
`difference_incremenet`: difference increment for the forward difference approximation  
`zeta`: a stabilization parameter for the continuation transformation   
`max_dim_krylov`: the maximam number of the iteration of the GMRES  

In [ ]:
T_f = 2.0
alpha = 1.0
horizon_division_num = 100
difference_increment = 1.0e-08
zeta = 1000
max_dim_krylov = 10

solver_params = solverparams.SolverParams(T_f, alpha, horizon_division_num, difference_increment, zeta, max_dim_krylov)

## Set parameters for the initialization of the solution  
`initial_guess_solution`: the initial guess of the solution  
`convergence_radius`: the convergence radius of Newton's method for the initialization of the solution  
`maximum_itr_newton`: maximum number of iterations in Newton's method for the initizalization of the solution  
`initial_guess_lagrange_multiplier`: the initial guess of the lagrange multiplier for the condensed constraints with respect to the saturation on the function of the control input

In [ ]:
initial_guess_solution = [0.01, 0.01, 0.01]
convergence_radius = 1.0e-06
maximum_itr_newton = 50

if solver_index == 1 or solver_index == 2:
    initialization_params = solverparams.InitializationParams(initial_guess_solution, convergence_radius, maximum_itr_newton)
else:
    initialization_params = solverparams.InitializationParams(initial_guess_solution, convergence_radius, maximum_itr_newton, initial_guess_lagrange_multiplier)

## Set parameters for numerical simulation
`initial_time`: initial time of the numerical simulation  
`initial_state`: initial state vector of the system  
`simulation_time`: simulation time of the numerical simulation  
`sampling_time`: the sampling time of the numerical simulation

In [ ]:
initial_time = 0  
initial_state = [0, 0, 0, 0]  
simulation_time = 10
sampling_time = 0.001  

simulation_params = simparams.SimulationParams(initial_time, initial_state, simulation_time, sampling_time)

## Generate main.cpp and CMakeLists.txt

In [ ]:
if solver_index == 1 or solver_index == 2:
    gencpp.generateMain(solver_index, solver_params, initialization_params, simulation_params, model_name)
else:
    gencpp.generateMain(solver_index, solver_params, initialization_params, simulation_params, model_name, saturation_list)    

gencpp.generateCMake(solver_index, model_name)
gencpp.generateCMakeForModel(model_name)

## Build and run simulation
NOTE: if you use Windows OS and an error occurs in `cppexec.setCMake(model_name)`, you may solve that error by running codes which are commented out instead of the original codes

In [ ]:
cppexec.setCMake(model_name)
cppexec.makeAndRun(model_name)
# cppexec.removeBuildDir(model_name)
# cppexec.setCMake(model_name, MSYS=True)
# cppexec.makeAndRun(model_name)

## Plot the simulation results

In [ ]:
plot = simplot.SimulationPlottor(model_name)
plot.setScales(2,5,2)
# plot.showPlots()
plot.savePlots()

## Draw an animation of the simulation results

In [ ]:
animgen = animgen.CartPole(model_name)
animgen.setSkipFrames(10)
animgen.generateAnimation()